<a href="https://colab.research.google.com/github/riyabhatnagar021/Movie_Review_Type/blob/master/Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
z=zipfile.ZipFile('drive/My Drive/project_data/Word-Embeddding-Sentiment-Classification-master.zip','r')
z.extractall()

In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.DataFrame()
df=pd.read_csv('Word-Embeddding-Sentiment-Classification-master/movie_data.csv',encoding='utf-8')
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [0]:
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [0]:
df['sentiment'][25000]
df['sentiment'][24999]

0

It means sentiment from 0-24999 is 0 and 25000 onwards is 1 

In [0]:
X=df['review']
y=df['sentiment']


In [0]:
X.shape

(50000,)

In [0]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tkn=Tokenizer()
c=tkn.fit_on_texts(X)
c

In [0]:

from keras.preprocessing.sequence import pad_sequences

In [0]:
max_length=100
vocab_size=len(tkn.word_index)+1
x_tokens=tkn.texts_to_sequences(X)
x_pad=pad_sequences(x_tokens,maxlen=max_length,padding='post')


In [0]:
vocab_size

124253

KERAS working

In [0]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding

Pretrained weights of Glove Embedding (glove6b100dtxt.zip)


In [0]:
a=zipfile.ZipFile('drive/My Drive/project_data/glove6b100dtxt.zip','r')
a.extractall()

In [0]:
import os

In [0]:
embeddings_index = {}
f = open(os.path.join('glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
# Embedding matrix
EMBEDDING_DIM=100
embedding_matrix = np.zeros((len(tkn.word_index) + 1, EMBEDDING_DIM))
for word, i in tkn.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
# Embedding Layer
embedding_layer = Embedding(len(tkn.word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

W0822 12:22:27.849471 140246914889600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Model Building

In [0]:
model=Sequential()
model.add(embedding_layer)
model.add(LSTM(units=32,dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


W0822 12:22:27.893616 140246914889600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 12:22:27.903962 140246914889600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 12:22:27.914556 140246914889600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0822 12:22:27.915448 140246914889600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0822 12:22:31.039756 140246914889600 depre

In [0]:
from sklearn.model_selection import train_test_split

In [0]:

X_train,X_test,y_train,y_test=train_test_split(x_pad,y,test_size=0.3)

In [0]:
model.fit(X_train,y_train,batch_size=128,epochs=25,validation_data=(X_test,y_test),verbose=2)

Train on 35000 samples, validate on 15000 samples
Epoch 1/25
 - 48s - loss: 0.3329 - acc: 0.8513 - val_loss: 0.2951 - val_acc: 0.8714
Epoch 2/25
 - 47s - loss: 0.3273 - acc: 0.8565 - val_loss: 0.3179 - val_acc: 0.8607
Epoch 3/25
 - 47s - loss: 0.3198 - acc: 0.8601 - val_loss: 0.2976 - val_acc: 0.8673
Epoch 4/25
 - 48s - loss: 0.3190 - acc: 0.8600 - val_loss: 0.2982 - val_acc: 0.8717
Epoch 5/25
 - 47s - loss: 0.3119 - acc: 0.8650 - val_loss: 0.3015 - val_acc: 0.8721
Epoch 6/25
 - 47s - loss: 0.3090 - acc: 0.8657 - val_loss: 0.2968 - val_acc: 0.8711
Epoch 7/25
 - 49s - loss: 0.3047 - acc: 0.8680 - val_loss: 0.3044 - val_acc: 0.8663
Epoch 8/25
 - 47s - loss: 0.3020 - acc: 0.8689 - val_loss: 0.3039 - val_acc: 0.8664
Epoch 9/25
 - 47s - loss: 0.3036 - acc: 0.8669 - val_loss: 0.3010 - val_acc: 0.8703
Epoch 10/25
 - 48s - loss: 0.2989 - acc: 0.8709 - val_loss: 0.2984 - val_acc: 0.8697
Epoch 11/25
 - 49s - loss: 0.2963 - acc: 0.8725 - val_loss: 0.2985 - val_acc: 0.8701
Epoch 12/25
 - 48s - los

Testing with some random reviews

In [0]:
test_sample1='This movie is fantastic ! I really like it because it is so good'
test_sample2='Can I get my money back please?'
test_sample3='This movie is great'
test_sample4='This movie is atrocious'
test_sample5='This movie is awesome.'

test_samples=[test_sample1,test_sample2,test_sample3,test_sample4,test_sample5]
test_sample_token=tkn.texts_to_sequences(test_samples)

test_sample_tokens_pad=pad_sequences(test_sample_token,maxlen=max_length)
ans=model.predict(x=test_sample_tokens_pad)
ans
for i in range(len(ans)):
  if(ans[i]>0.5):
    print('Its a positive review')
  else:
    print('Its a negative review')

Its a positive review
Its a negative review
Its a positive review
Its a negative review
Its a positive review


saving trained weights

In [0]:
model.save_weights('senti_trained_wgts.h5')

In [0]:
# model.load_weights('drive/My Drive/project_data/senti_trained_wgts.h5')